In [1]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
import xlrd
from py2neo import Graph,Node,Relationship
from py2neo.database import ClientError
import re

In [2]:
# option_entities_file = 'entities.csv'
# option_relations_file = 'relations.txt'
# option_properties_file = 'properties.txt'
option_deleteAllFlag = 1
option_demoFlag = 1
option_read_csv_row =  -1 # -1 for all rows, positive number k for first k rows.
option_userName = 'neo4j'
option_passWord = '123123'
option_config_file = r''
option_entities_dir = ''
option_column_prefix = "" 
option_label = ""
option_neo4j_ip = "localhost"
option_neo4j_port = "7474"
option_pas = False
option_intesrv = False
option_decoration = True
option_remote = False

if option_pas == True:
#     option_config_file = r'E:/天远/知识图谱/天远知识图谱文档/数据血缘/PAS/PAS_config.xlsx'
    option_config_file = r'./config/PAS_config.xlsx'
    option_entities_dir = './data/pas/'
    option_column_prefix = "" 
    option_label = "数据起源"

if option_intesrv == True:
    option_config_file = r'E:/天远/知识图谱/天远知识图谱文档/天远百度/config/config_intesrv/config_v2.2.0.xlsx'
#     option_config_file = './config1.3.xlsx'
    option_entities_dir = './data/intesrv_real/'
    option_column_prefix = "plt_" 
    option_label = "智能服务"
    
if option_decoration == True:
    option_config_file = r'E:/天远/知识图谱/天远知识图谱文档/天远百度/config/config_manual/config_manual_v1.3.1.xlsx'
#     option_config_file = './config1.3.xlsx'
    option_entities_dir = './data/decoration/'
    option_column_prefix = "" 
    option_label = "装修手册"
    
if option_remote == True:
    option_passWord = '123456'
    option_neo4j_ip = "101.6.15.212"


##### debug

## 读取配置表

### 主外键对应表

In [3]:
df = pd.read_excel(open(option_config_file, 'rb'),sheet_name='主外键对应表')
for col in df.columns:
    df[col] = df[col].astype(str).str.lower()
df['primary_table'] = df['主键'].apply(lambda x: x.split('.')[0])
df['primary_key'] = option_column_prefix + df['主键'].apply(lambda x: x.split('.')[1])
df['foreign_table'] = df['外键'].apply(lambda x: x.split('.')[0])
df['foreign_key'] = option_column_prefix + df['外键'].apply(lambda x: x.split('.')[1])
df_foreign = df
df_foreign

,主键,外键,primary_table,primary_key,foreign_table,foreign_key


#### 根据主外键关系，对表进行拓扑排序

In [4]:
if option_decoration == True:
    tables_name = ['manual_fc']
else:
    tables_name = []
    df_foreign_tobedeleted = df_foreign.copy()
    count = 0
    while len(df_foreign_tobedeleted) > 0:
    #     print(count)
        a = df_foreign_tobedeleted['primary_table'].unique()
        b = df_foreign_tobedeleted['foreign_table'].unique()
        c = np.concatenate((a,b))
        tables_name_unordered = set(c)
    #     print(tables_name_unordered)
        foreign_table = set(df_foreign_tobedeleted['foreign_table'])
        table = tables_name_unordered.difference(foreign_table).pop()
    #     print(tables_name_unordered.difference(foreign_table))
        tables_name.append(table)
    #     print(table)
        index_tobedeleted = df_foreign_tobedeleted.loc[df_foreign_tobedeleted['primary_table'] == table].index
    #     print(index_tobedeleted)
        df_foreign_tobedeleted.drop(index_tobedeleted, inplace=True)
    #     print(df_foreign_tobedeleted[['primary_table','foreign_table']])
        count += 1    

    df_foreign_tobedeleted = df_foreign.copy()
    a = df_foreign_tobedeleted['primary_table'].unique()
    b = df_foreign_tobedeleted['foreign_table'].unique()
    c = np.concatenate((a,b))
    tables_name_unordered = set(c)
    table_not_added = tables_name_unordered.difference(set(tables_name))
    while len(table_not_added) > 0:
        tables_name.append(table_not_added.pop())
    tables_name

##### debug

### 关系表

In [5]:
dict_relation = {"关系主语":"sub",
                   "关系谓语":"predicate",
                   "关系宾语":"obj",
                   "关系类型":"type",
                   "关系所在数据库表":"table",
                    "关系属性":"prop_ch_str",
                    "关系属性对应数据库属性":"prop_col_str"}
dict_relation_reverse = {value:key for key,value in dict_relation.items()}

df = pd.read_excel(open(option_config_file, 'rb'),sheet_name='关系表')
for col in df.columns:
    df[col] = df[col].astype(str).str.lower()
    df[col] = df[col].str.replace("\n","")
df = df.rename(index=str, columns=dict_relation)
df['prop_ch'] = df['prop_ch_str'].astype(str).apply(lambda x: [y for y in re.split(",|，", x) if x != "nan"] )
df['prop_col'] = df['prop_col_str'].astype(str).apply(lambda x: [option_column_prefix+y for y in re.split(",|，", x) if x != "nan"] )
df_relation = df
df_relation

,id,sub,predicate,obj,table,type,prop_ch_str,prop_col_str,prop_ch,prop_col
0,1,故障,包含,故障内容,manual_fc,nan,nan,nan,[],[]
1,2,故障,包含,相关信息,manual_fc,nan,nan,nan,[],[]
2,3,故障,导致,机器发生的故障,manual_fc,nan,nan,nan,[],[]
3,4,故障,表现为,控制器的反应,manual_fc,nan,nan,nan,[],[]
4,5,故障,包含,相关图片,nan,暂无,nan,nan,[],[]
5,6,故障,产生于,部件,manual_fc,nan,nan,nan,[],[]
6,7,故障,状态为,故障状态,nan,暂无,nan,nan,[],[]
7,8,故障,因为,故障起因,manual_fc,nan,nan,nan,[],[]
8,9,故障起因,发生于,部件,manual_fc,nan,nan,nan,[],[]
9,10,故障起因,参见,故障,nan,暂无,nan,nan,[],[]


### 概念表

In [6]:
def split_compound_table(paragraph):
    print(paragraph)
    results = []
    for sentence in re.split(",|，", paragraph):        
        compound = {}
        compound['main'] = sentence.split('-')[0]
        compound['foreign'] = sentence.split('-')[1]
        compound['main_table'] = compound['main'].split('.')[0]
        compound['main_col'] = option_column_prefix + compound['main'].split('.')[1]
        compound['foreign_table'] = compound['foreign'].split('.')[0]
        compound['foreign_col'] = option_column_prefix + compound['foreign'].split('.')[1]
        results.append(compound)
    return results
# split_compound_table(s+","+s)

In [7]:
dict_concept = {"模式图中概念":"concept_ch",
                   "显示名":"display_name",
                   "数据库表/属性":"table_and_column",
                   "类型":"type",
                   "逻辑代码连接":"code",
                   "内部属性":"inner_properties_ch",
                   "内部属性对应数据库属性":"properties_col_str",
                   "逻辑代码所需数据库表":"compound_table_str"}
dict_concept_reverse = {value:key for key,value in dict_concept.items()}

df = pd.read_excel(open(option_config_file, 'rb'),sheet_name='模式图的概念与数据库对应表')
for col in df.columns:
    df[col] = df[col].astype(str).str.lower()
    df[col] = df[col].str.replace("\n","")
df = df.rename(index=str, columns=dict_concept)
# print(df)
df['table'] = None
df['col'] = None
# df['test'] = df['concept_ch'].apply(split_compound_table)
# print(df)
# df[df['type'] == '直接']['table'] = df[df['type'] == '直接']['table_and_column'].astype(str).apply(lambda x: x.split('.')[0])
df.loc[df['type'] == '直接','table'] = df[df['type'] == '直接']['table_and_column'].astype(str).apply(lambda x: x.split('.')[0])
df.loc[df['type'] == '直接','col'] = option_column_prefix + df[df['type'] == '直接']['table_and_column'].astype(str).apply(lambda x: x.split('.')[1])
df.loc[df['type'] == '直接','properties_col'] = df[df['type'] == '直接']['properties_col_str'].astype(str).apply(lambda x: [option_column_prefix+y for y in re.split(",|，", x) if x != "nan"] )
df.loc[df['type'] == '直接','properties_col_ch'] = df[df['type'] == '直接']['inner_properties_ch'].astype(str).apply(lambda x: [y for y in re.split(",|，", x) if x != "nan"] )
df.loc[(df['type'] == '合成概念'), 'table'] = df[(df['type'] == '合成概念')]['table_and_column'].astype(str)
df.loc[(df['type'] == '合成概念') & (df['compound_table_str'] != 'nan'), 'compound_table'] = df[(df['type'] == '合成概念') & (df['compound_table_str'] != 'nan')]['compound_table_str'].astype(str).apply(split_compound_table)
df_concept = df
df_concept

,concept_ch,display_name,table_and_column,type,inner_properties_ch,properties_col_str,code,compound_table_str,所属域,table,col,properties_col,properties_col_ch,compound_table
0,故障,故障名称,manual_fc.faultcode,直接,"故障名称,报警级别","faultname,alertlevel",nan,nan,故障诊断,manual_fc,faultcode,"[faultname, alertlevel]","[故障名称, 报警级别]",NaN
1,故障内容,unique_id,manual_fc.faultcontent,直接,nan,nan,nan,nan,故障诊断,manual_fc,faultcontent,[],[],NaN
2,机器发生的故障,unique_id,manual_fc.machinefault,直接,nan,nan,nan,nan,故障诊断,manual_fc,machinefault,[],[],NaN
3,控制器的反应,unique_id,manual_fc.controllerreaction,直接,nan,nan,nan,nan,故障诊断,manual_fc,controllerreaction,[],[],NaN
4,相关信息,unique_id,manual_fc.relatedinfo,直接,nan,nan,nan,nan,故障诊断,manual_fc,relatedinfo,[],[],NaN
5,故障起因,unique_id,manual_fc.cause,直接,nan,nan,nan,nan,故障诊断,manual_fc,cause,[],[],NaN
6,诊断操作,unique_id,manual_fc.diagnosisaction,直接,nan,nan,nan,nan,故障诊断,manual_fc,diagnosisaction,[],[],NaN
7,相关图片,unique_id,manual_fc.image,暂无,nan,nan,nan,nan,故障诊断,None,None,NaN,NaN,NaN
8,标准值,id,manual_fc.standardval,暂无,"id,测量位置，限定类型，标准值类型，min单位，max单位，min，max，限定条件取值，...",nan,nan,nan,故障诊断,None,None,NaN,NaN,NaN
9,检查系统,unique_id,manual_fc.monitorsystem,直接,nan,nan,nan,nan,故障诊断,manual_fc,monitorsystem,[],[],NaN


#### debug

## 读取实体数据

In [8]:
tables = {}
total_length = 0
for table_name in tables_name:
    print("start to access table " + table_name)
    path = option_entities_dir + table_name + ".csv"
    df = pd.read_csv(path,sep=';',encoding='utf-16')
    for key in list(df.columns.values):  # 如果不这样的话，可能会读入int64的数字，但是neo4j不支持int64
        df[key] = df[key].astype("object")
    print(df.shape) 
    total_length += df.shape[0]
    tables[table_name] = df[:option_read_csv_row]
    print("end to access table " + table_name)
# tables
total_length # 60460

start to access table manual_fc
(2519, 15)
end to access table manual_fc


2519

## 导入Neo4j

In [9]:
graph = Graph('http://%s:%s'%(option_neo4j_ip,option_neo4j_port),username=option_userName,password=option_passWord)

In [10]:
if option_deleteAllFlag == 1:
    # 只删除本个label域下的节点和关系
    graph.run("MATCH (n:%s) OPTIONAL MATCH (n)-[r]-() DELETE r,n"%(option_label)) 
    graph.delete_all()


### 模式图

In [11]:
df_concept.head()

,concept_ch,display_name,table_and_column,type,inner_properties_ch,properties_col_str,code,compound_table_str,所属域,table,col,properties_col,properties_col_ch,compound_table
0,故障,故障名称,manual_fc.faultcode,直接,"故障名称,报警级别","faultname,alertlevel",nan,nan,故障诊断,manual_fc,faultcode,"[faultname, alertlevel]","[故障名称, 报警级别]",NaN
1,故障内容,unique_id,manual_fc.faultcontent,直接,nan,nan,nan,nan,故障诊断,manual_fc,faultcontent,[],[],NaN
2,机器发生的故障,unique_id,manual_fc.machinefault,直接,nan,nan,nan,nan,故障诊断,manual_fc,machinefault,[],[],NaN
3,控制器的反应,unique_id,manual_fc.controllerreaction,直接,nan,nan,nan,nan,故障诊断,manual_fc,controllerreaction,[],[],NaN
4,相关信息,unique_id,manual_fc.relatedinfo,直接,nan,nan,nan,nan,故障诊断,manual_fc,relatedinfo,[],[],NaN


In [12]:
nodes_concept = {}
for index in tqdm(list(df_concept.index)):
    index = int(index)
#     print(index)
#     print(df_concept.iloc[index]['concept_ch'])
    key = df_concept.iloc[index]['concept_ch']
    nodes_concept[key] = Node(key)
    nodes_concept[key].add_label("概念节点")
    if option_label != "":
        nodes_concept[key].add_label(option_label)
        nodes_concept[key]['domain_name'] = option_label
    # 为概念节点增加属性，把该有的东西都放进去
    for col in list(df_concept.columns.values):
        col_chinese = dict_concept_reverse[col] if col in dict_concept_reverse.keys() else col
        nodes_concept[key][col_chinese] = str(df_concept[col][index]) if df_concept[col][index] == df_concept[col][index] else ""
#     print(nodes_concept[key])
    graph.create(nodes_concept[key])

100%|██████████████████████████████████████████████████████████████████████████████████| 13/13 [00:00<00:00, 21.94it/s]


In [13]:
df_relation.head()

,id,sub,predicate,obj,table,type,prop_ch_str,prop_col_str,prop_ch,prop_col
0,1,故障,包含,故障内容,manual_fc,nan,nan,nan,[],[]
1,2,故障,包含,相关信息,manual_fc,nan,nan,nan,[],[]
2,3,故障,导致,机器发生的故障,manual_fc,nan,nan,nan,[],[]
3,4,故障,表现为,控制器的反应,manual_fc,nan,nan,nan,[],[]
4,5,故障,包含,相关图片,nan,暂无,nan,nan,[],[]


In [14]:
for index in tqdm(list(df_relation.index)):
    index = int(index)
#     print(index)
    sub = df_relation.iloc[index]['sub']
    obj = df_relation.iloc[index]['obj']
#     try:
#     print(sub)
#     print(obj)
    aRelation = Relationship(nodes_concept[sub], df_relation.iloc[index]['predicate'], nodes_concept[obj] )
    aRelation["id_relation"] = df_relation.iloc[index]['id']
    for col in list(df_relation.columns.values):
        col_chinese = dict_relation_reverse[col] if col in dict_relation_reverse.keys() else col
        aRelation[col_chinese] = df_relation[col][index] if df_relation[col][index] == df_relation[col][index] else ""
     
    graph.create(aRelation)
#     except(KeyError):
#         pass

100%|██████████████████████████████████████████████████████████████████████████████████| 17/17 [00:00<00:00, 21.00it/s]


### 合成概念函数test

In [15]:
def test1(df):
#     print(df)
    return df['plt_lastmodifytime'][0] + " and " + df['plt_createtime'][0]

In [16]:
def test2(df):
#     print(df)
    return len(df)

In [17]:
def test3(df):
#     print(df)
    return len(df)

### 数据图

#### 单个表数据(debug)

#### 所有数据

In [18]:
def change2float(x):
    if re.match("^\d+?(\.\d+?)?$", str(x)) is None:
        return x
    else:
        return float(x)
change2float('1.09')

1.09

In [19]:
def transferOneRowToNeo4j():
    def establishNode():
        def establishDirectNode():
            pass
        def establishCompoundNode():
            pass
        establishDirectNode()
        establishCompoundNode()
        pass
    def establishRelation():
        def retrieveNode():
            pass
        def establishRelationWithNode():
            pass
        retrieveNode()
        establishRelationWithNode()
        pass
    establishNode()
    establishRelation()
    pass
transferOneRowToNeo4j()
# 考虑换成class，一个public函数，其余函数都是private

In [20]:
for table_name in tables_name:
    print(table_name)
    # 取出属于本表的几个小表，包括实体数据表、概念表、关系表
    df_table = tables[table_name]
    df_concept_one_table = df_concept[df_concept['table'] == table_name]
    df_relation_one_table = df_relation[df_relation['table'] == table_name]

    concept_ch_list_one_table = list(df_concept_one_table['concept_ch'])
    for i_row in tqdm(range(0,df_table.shape[0])): 
        transferOneRowToNeo4j()

manual_fc


100%|█████████████████████████████████████████████████████████████████████████| 2518/2518 [00:00<00:00, 1262855.13it/s]


In [21]:
# if option_deleteAllFlag == 1:
#     graph.delete_all()

# table_name = 'plt_intesrv_tydevice'
for table_name in tables_name:
    print(table_name)
    # 取出属于本表的几个小表，包括实体数据表、概念表、关系表
    df_table = tables[table_name]
    df_concept_one_table = df_concept[df_concept['table'] == table_name]
    df_relation_one_table = df_relation[df_relation['table'] == table_name]

    concept_ch_list_one_table = list(df_concept_one_table['concept_ch'])
    # 对于数据表的每行进行处理
    for i_row in tqdm(range(0,df_table.shape[0])): 
#         print(i_row)
        entity_node = {}
        # 对于概念表中的每个概念进行处理
        for i_entity in list(df_concept_one_table.index):
            key = df_concept_one_table['concept_ch'][i_entity] # eg:"客户","挖掘机"
            if df_concept_one_table['type'][i_entity] == '直接':
                value = df_table[ df_concept_one_table['col'][i_entity] ][i_row] # eg:"张三","PC200"
                if value == value: # 如果该节点的值非空
                    nodeBefore = graph.nodes.match(key, value=value).first()
                    if nodeBefore is None:
                        aNode = Node(key)
                        aNode['value'] = change2float(value)
                        aNode['type'] = key
                        aNode.add_label("实例节点") # Neo4j的官方label，支持高效索引

                        # 为该节点增加对应属性
                        assert len(df_concept_one_table['properties_col'][i_entity]) == len(df_concept_one_table['properties_col_ch'][i_entity]) \
                            , "properties_col=%s, properties_col_ch=%s. They have different length."%(str(df_concept_one_table['properties_col'][i_entity]), str(df_concept_one_table['properties_col_ch'][i_entity]))
                        prop_num = len(df_concept_one_table['properties_col'][i_entity])
                        for i_prop in range(0,prop_num):
                            prop_key = df_concept_one_table['properties_col'][i_entity][i_prop]  # eg: prop_key = 'plt_devicebrand'
                            prop_key_ch = df_concept_one_table['properties_col_ch'][i_entity][i_prop] # eg: prop_key_ch = '品牌'
                            prop_value = df_table[prop_key][i_row]
                            if prop_value == prop_value: 
                                aNode[prop_key_ch] = change2float(prop_value)
                            else: # 属性值如果为空的话，会置为空字符串
                                aNode[prop_key_ch] = ''

                        # 为该节点增加display_name属性
    #                     display = "(" + str(key) + ")"
                        display = str(key) + ":"
                        display_col = df_concept_one_table['display_name'][i_entity]
                        if display_col == 'unique_id':
                            display += value 
                        elif display_col == "nan": # 如果那一列为空的话，display就只显示概念名，不显示具体指
                            display_col = display_col[:-2]
                        else:
                            # 根据中文列名找到数据库列名
                            prop_num = len(df_concept_one_table['properties_col'][i_entity])
                            for i_prop in range(0,prop_num):
                                prop_key = df_concept_one_table['properties_col'][i_entity][i_prop]  # eg: prop_key = 'plt_devicebrand'
                                prop_key_ch = df_concept_one_table['properties_col_ch'][i_entity][i_prop] # eg: prop_key_ch = '品牌'
                                if prop_key_ch == display_col:
                                    break
                            assert i_prop != prop_num - 1 or (i_prop == prop_num - 1 and prop_key_ch == display_col) \
                                , "display_col=%s, properties_col_ch=%s. display_col doesn't exist in properties_col_ch."%(display_col, str(df_concept_one_table['properties_col_ch'][i_entity]))
                            display += str(df_table[prop_key][i_row])
    #                     print("debug:display=", display)
                        aNode['display_name'] = display
                        if option_label != "":
                            aNode.add_label(option_label)
                            aNode['domain_name'] = option_label
    #                     print(aNode)
                        graph.create(aNode)
                    else:
                        aNode = nodeBefore
                    entity_node[key] = aNode
            # 生成合成概念实体
            elif df_concept_one_table['type'][i_entity] == '合成概念':
#                 print(df_concept_one_table['concept_ch'][i_entity])
#                 print(df_concept_one_table['compound_table'][i_entity])
                if str(df_concept_one_table['compound_table'][i_entity]) == 'nan': # 非空的话只需要把当前所在行传入函数
                    indexs = [] 
                    indexs.append(i_row)
                    # use list as index to return DataFrame instead of Series though the list has only one element
                    df_compound_need = df_table.loc[indexs,:].reset_index(drop=True) 
                else: # 处理联合多表的情况
                    compound = df_concept_one_table['compound_table'][i_entity][0]
                    assert table_name == compound['main_table'] # 第一个跳转表的主表应该得是本表
                    main_key_value = df_table[compound['main_col']][i_row]
                    df_table_new = tables[compound['foreign_table']]
                    foreign_col = compound['foreign_col']
                    df_compound_need = df_table_new.loc[df_table_new[foreign_col] == main_key_value]
                    # TODO:处理联合多表的情况
                value = eval(df_concept_one_table['code'][i_entity] + "(df_compound_need)")
                aNode = Node(key)
                aNode['value'] = change2float(value)
                aNode['type'] = key
                aNode['display_name'] = str(key) + ":" + str(value) 
                if option_label != "":
                    aNode.add_label(option_label)
                    aNode['domain_name'] = option_label
                graph.create(aNode)
                entity_node[key] = aNode
    #     print(entity_node)
    
        # 建立本行数据中的节点间的关系
        for i_relation in list(df_relation_one_table.index):
    #         try: 
            # 建立sub主语实体，有可能已经在entity_node中，也有可能是外键
            entity_ch = df_relation_one_table['sub'][i_relation]
            if entity_ch in entity_node.keys(): # 本行数据确实有该节点
                entity = entity_node[entity_ch]
            elif entity_ch not in concept_ch_list_one_table: # 本行数据无该节点，则为外键
                # entity_ch(eg:客户)去模式表里找该概念的数据库主键来源，根据主键和当前表table_name来找到该概念在当前表的外键列名，然后根据该外键列名找到外键具体值，去neo4j里查询该节点，
#                 print(entity_ch)
                primary_key_table = df_concept.loc[df_concept['concept_ch'] == entity_ch, 'table'].item()
                primary_key_col = df_concept.loc[df_concept['concept_ch'] == entity_ch, 'col'].item()
#                 print(primary_key_table)
#                 print(primary_key_col)
#                 print(entity_ch)
#                 print(df_foreign['primary_table'] == primary_key_table)
#                 print((df_foreign['primary_key'] == primary_key_col))
#                 print((df_foreign['foreign_table'] == table_name))
                foreign_key = df_foreign.loc[(df_foreign['primary_table'] == primary_key_table)
                                  & (df_foreign['primary_key'] == primary_key_col)
                                  & (df_foreign['foreign_table'] == table_name), 'foreign_key'].item()
#                 print(foreign_key)
                key = entity_ch
                value = df_table[foreign_key][i_row]
                if value != value: # 虽然该键是外键，但是该键依然可能是控制(玄学数据)
                    continue
                nodeBefore = graph.nodes.match(key, value=value).first()
    #             assert nodeBefore is not None
                if nodeBefore is None: # 这是测试数据的锅，否则的话永远不应该为None的
                    continue
                entity = nodeBefore
    #             print(type(primary_key_table)) 
    #             primary_key_table, primary_key_col
    #             continue
            else: # 本行数据的该节点value为空，所以没有该节点
                continue
            sub = entity

            # 建立obj宾语实体，这二三十航代码都是抄的上面的，只要把首末行的sub改成obj就可以了
            entity_ch = df_relation_one_table['obj'][i_relation]
            if entity_ch in entity_node.keys(): # 本行数据确实有该节点
                entity = entity_node[entity_ch]
            elif entity_ch not in concept_ch_list_one_table: # 本行数据无该节点，则为外键
                # 根据entity_ch(eg:客户)去模式表里找该概念的数据库主键来源，根据主键和当前表table_name来找到该概念在当前表的外键列名，然后根据该外键列名找到外键具体值，去neo4j里查询该节点，
#                 print(entity_ch)
                primary_key_table = df_concept.loc[df_concept['concept_ch'] == entity_ch, 'table'].item()
                primary_key_col = df_concept.loc[df_concept['concept_ch'] == entity_ch, 'col'].item()
#                 print(primary_key_table)
#                 print(primary_key_col)
#                 print(entity_ch)
#                 print(df_foreign['primary_table'] == primary_key_table)
#                 print((df_foreign['primary_key'] == primary_key_col))
#                 print((df_foreign['foreign_table'] == table_name))
#                 print("-"*10)
#                 print(df_foreign.loc[(df_foreign['primary_table'] == primary_key_table)
#                                   & (df_foreign['primary_key'] == primary_key_col)
#                                   & (df_foreign['foreign_table'] == table_name), 'foreign_key'])
#                 print("-"*10)
                foreign_key = df_foreign.loc[(df_foreign['primary_table'] == primary_key_table)
                                  & (df_foreign['primary_key'] == primary_key_col)
                                  & (df_foreign['foreign_table'] == table_name), 'foreign_key'].item()
#                 print(foreign_key)
                key = entity_ch
                value = df_table[foreign_key][i_row]
                if value != value: # 虽然该键是外键，但是该键依然可能是空值(玄学数据)
                    continue
                nodeBefore = graph.nodes.match(key, value=value).first()
    #             assert nodeBefore is not None
                if nodeBefore is None: # 测试数据的锅，否则的话永远不应该为None的
                    continue            
                entity = nodeBefore
    #             print(type(primary_key_table))
    #             primary_key_table, primary_key_col
    #             continue
            else: # 本行数据的该节点value为空，所以没有该节点
                continue
            obj = entity

            # 建立主宾实体间的关系
            predicate = df_relation_one_table['predicate'][i_relation]
            aRelation = Relationship(sub, predicate, obj)
            aRelation['id_relation'] = df_relation_one_table['id'][i_relation]
            # 为关系增加属性
            assert len(df_relation_one_table['prop_ch'][i_relation]) == len(df_relation_one_table['prop_col'][i_relation]) \
                , "prop_ch=%s, prop_col=%s. They have different length."%(str(df_relation_one_table['prop_ch'][i_relation]), str(df_relation_one_table['prop_col'][i_relation]))
            prop_num = len(df_relation_one_table['prop_ch'][i_relation])
            for i_prop in range(0,prop_num):
                prop_key = df_relation_one_table['prop_col'][i_relation][i_prop]
                prop_key_ch = df_relation_one_table['prop_ch'][i_relation][i_prop]
                prop_value = df_table[prop_key][i_row]
                if prop_value == prop_value:
                    aRelation[prop_key_ch] = change2float(prop_value)
                else:
                    aRelation[prop_key_ch] = ''

            graph.create(aRelation)
    #         except KeyError: # 因为某些节点可能因为值为空所以没有该节点，所以在entity_node字典中没有该节点
    #             pass
    #         print(entity_name)

manual_fc


100%|██████████████████████████████████████████████████████████████████████████████| 2518/2518 [03:55<00:00, 10.68it/s]
